In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

## Player map

In [4]:
df_player_map = pd.read_excel('SFBB-Player-ID-Map.xlsx')

In [5]:
df_player_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   IDPLAYER        2606 non-null   object        
 1   PLAYERNAME      2606 non-null   object        
 2   BIRTHDATE       2605 non-null   datetime64[ns]
 3   FIRSTNAME       2606 non-null   object        
 4   LASTNAME        2606 non-null   object        
 5   TEAM            1618 non-null   object        
 6   LG              1618 non-null   object        
 7   POS             2606 non-null   object        
 8   IDFANGRAPHS     2600 non-null   object        
 9   FANGRAPHSNAME   2604 non-null   object        
 10  MLBID           2605 non-null   float64       
 11  MLBNAME         2604 non-null   object        
 12  CBSID           2554 non-null   float64       
 13  CBSNAME         2555 non-null   object        
 14  RETROID         2226 non-null   object        
 15  BREF

In [6]:
df_player_map[df_player_map['MLBNAME'] == 'Max Scherzer']['MLBID']

2129    453286.0
Name: MLBID, dtype: float64

## Selenium

In [7]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [8]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2021%7C2020%7C2019%7C2018%7C2017%7C2016%7C2015%7C2014%7C2013%7C2012%7C2011%7C2010%7C2009%7C2008%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&hfFlag=&hfBBT=&pitchers_lookup%5B%5D=453286&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&min_pas=0#results")

In [9]:
driver.find_element_by_id('id_453286').click()

In [13]:
soup = BeautifulSoup(driver.page_source)

In [14]:
table = soup.find('table',id="ajaxTable_453286")

In [15]:
rows = table.find_all('tr')

In [16]:
rows[1].find_all('td')[13]

<td style="max-width: 250px !important;  white-space: normal !important;">Wil Myers triples (2) on a line drive to left fielder AJ Pollock, deflected by center fielder Gavin Lux.   Victor Caratini scores.</td>

In [17]:
columns = ['Pitch','MPH','EV','Pitcher','Batter','Dist','Spin Rate','LA','Zone','Date','Count','Inning',
           'Pitch Result','PA Result']
df = pd.DataFrame(columns=columns)
row_idx = 0
for row in rows[1:]:
    cols = row.find_all('td')
    df.loc[row_idx] = [
        cols[0].text.strip(),
        cols[1].text.strip(),
        cols[2].text.strip(),
        cols[3].text.strip(),
        cols[4].text.strip(),
        cols[5].text.strip(),
        cols[6].text.strip(),
        cols[7].text.strip(),
        cols[8].text.strip(),
        cols[9].text.strip(),
        cols[10].text.strip(),
        cols[11].text.strip(),
        cols[12].text.strip(),
        cols[13].text.strip(),
    ]
    row_idx += 1

In [18]:
df

,Pitch,MPH,EV,Pitcher,Batter,Dist,Spin Rate,LA,Zone,Date,Count,Inning,Pitch Result,PA Result
0,FF,94.3,98.8,"Scherzer, Max","Myers, Wil",376,2486,23,1,2021-09-29,0-2,Top 6,hit_into_play,Wil Myers triples (2) on a line drive to left ...
1,SL,84.8,,"Scherzer, Max","Myers, Wil",,2439,,4,2021-09-29,0-1,Top 6,swinging_strike,Wil Myers triples (2) on a line drive to left ...
2,SL,84.7,,"Scherzer, Max","Myers, Wil",,2365,,7,2021-09-29,0-0,Top 6,called_strike,Wil Myers triples (2) on a line drive to left ...
3,CU,76.4,81.9,"Scherzer, Max","Caratini, Victor",235,2790,20,8,2021-09-29,0-1,Top 6,hit_into_play,Victor Caratini singles on a line drive to rig...
4,CU,76.2,,"Scherzer, Max","Caratini, Victor",,2742,,14,2021-09-29,0-0,Top 6,called_strike,Victor Caratini singles on a line drive to rig...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,FF,96.0,,"Scherzer, Max","Hall, Bill",,,,14,2009-04-30,1-1,Bot 4,ball,Bill Hall pops out to first baseman Chad Tracy.
39996,FF,96.9,,"Scherzer, Max","Hall, Bill",,,,14,2009-04-30,0-1,Bot 4,ball,Bill Hall pops out to first baseman Chad Tracy.
39997,FF,94.7,,"Scherzer, Max","Hall, Bill",,,,2,2009-04-30,0-0,Bot 4,swinging_strike,Bill Hall pops out to first baseman Chad Tracy.
39998,FF,95.7,,"Scherzer, Max","Hardy, J.J.",,,,13,2009-04-30,0-1,Bot 4,hit_into_play,J.J. Hardy flies out to right fielder Justin U...


In [4]:
#40000 limit so need to scrape 2008 and 2009 
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get("https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2009%7C2008%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfInfield=&team=&position=&hfOutfield=&hfRO=&home_road=&hfFlag=&hfBBT=&pitchers_lookup%5B%5D=453286&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&min_pas=0#results")

In [5]:
driver.find_element_by_id('id_453286').click()

In [7]:
soup = BeautifulSoup(driver.page_source)
table = soup.find('table',id="ajaxTable_453286")
rows = table.find_all('tr')
columns = ['Pitch','MPH','EV','Pitcher','Batter','Dist','Spin Rate','LA','Zone','Date','Count','Inning',
           'Pitch Result','PA Result']
df_0809 = pd.DataFrame(columns=columns)
row_idx = 0
for row in rows[1:]:
    cols = row.find_all('td')
    df_0809.loc[row_idx] = [
        cols[0].text.strip(),
        cols[1].text.strip(),
        cols[2].text.strip(),
        cols[3].text.strip(),
        cols[4].text.strip(),
        cols[5].text.strip(),
        cols[6].text.strip(),
        cols[7].text.strip(),
        cols[8].text.strip(),
        cols[9].text.strip(),
        cols[10].text.strip(),
        cols[11].text.strip(),
        cols[12].text.strip(),
        cols[13].text.strip(),
    ]
    row_idx += 1

In [8]:
df_0809

,Pitch,MPH,EV,Pitcher,Batter,Dist,Spin Rate,LA,Zone,Date,Count,Inning,Pitch Result,PA Result
0,CH,82.1,,"Scherzer, Max","Headley, Chase",,,,11,2009-09-25,1-2,Top 5,swinging_strike,Chase Headley strikes out swinging.
1,SL,82.7,,"Scherzer, Max","Headley, Chase",,,,1,2009-09-25,1-1,Top 5,called_strike,Chase Headley strikes out swinging.
2,SL,82.4,,"Scherzer, Max","Headley, Chase",,,,5,2009-09-25,1-0,Top 5,foul,Chase Headley strikes out swinging.
3,FF,90.9,,"Scherzer, Max","Headley, Chase",,,,11,2009-09-25,0-0,Top 5,ball,Chase Headley strikes out swinging.
4,CH,83.9,,"Scherzer, Max","González, Adrián",,,,4,2009-09-25,3-2,Top 5,hit_into_play,Adrian Gonzalez singles on a line drive to rig...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,FF,97.9,,"Scherzer, Max","Cassel, Jack",,,,8,2008-04-29,2-2,Top 3,foul,Jack Cassel strikes out swinging.
3998,FF,97.1,,"Scherzer, Max","Cassel, Jack",,,,6,2008-04-29,2-1,Top 3,swinging_strike,Jack Cassel strikes out swinging.
3999,FF,96.5,,"Scherzer, Max","Cassel, Jack",,,,6,2008-04-29,2-0,Top 3,called_strike,Jack Cassel strikes out swinging.
4000,FF,96.1,,"Scherzer, Max","Cassel, Jack",,,,13,2008-04-29,1-0,Top 3,ball,Jack Cassel strikes out swinging.


In [12]:
df['Year'] = df['Date'].map(lambda X:X[:4]).astype('int')
df_0809['Year'] = df_0809['Date'].map(lambda X:X[:4]).astype('int')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    40000 non-null  int64  
 1   Pitch         39834 non-null  object 
 2   MPH           39671 non-null  float64
 3   EV            5259 non-null   float64
 4   Pitcher       40000 non-null  object 
 5   Batter        40000 non-null  object 
 6   Dist          5049 non-null   float64
 7   Spin Rate     19544 non-null  float64
 8   LA            5259 non-null   float64
 9   Zone          39666 non-null  float64
 10  Date          40000 non-null  object 
 11  Count         40000 non-null  object 
 12  Inning        40000 non-null  object 
 13  Pitch Result  40000 non-null  object 
 14  PA Result     40000 non-null  object 
 15  Year          40000 non-null  int64  
dtypes: float64(6), int64(2), object(8)
memory usage: 4.9+ MB


In [18]:
df = pd.concat([df[df['Year'] != 2009],df_0809])

In [20]:
#done
df.to_csv('scherzer_bs.csv')

In [19]:
df.groupby(['Year','Pitch']).count()

Unnamed: 0   MPH   EV  Pitcher  Batter  Dist  Spin Rate   LA  \
Year Pitch                                                                 
2008                 0    29   29       29      29    29         29   29   
     CH              0    95   95       95      95    95         95   95   
     FF              0   612  612      612     612   612        612  612   
     FT              0    35   35       35      35    35         35   35   
     IN              0     4    4        4       4     4          4    4   
...                ...   ...  ...      ...     ...   ...        ...  ...   
2021 CH            421   421  134      421     421   133        421  134   
     CU            277   277   56      277     277    56        276   56   
     FC            256   256  103      256     256   106        256  103   
     FF           1316  1316  428     1316    1316   429       1316  428   
     SL            551   551  145      551     551   145        551  145   

            Zone  Date  Count  Inning  Pitch Result  PA Result  
Year Pitch                                                      
2008          29    29     29      29            29         29  
     CH       95    95     95      95            95         95  
     FF      612   612    612     612           612        612  
     FT       35    35     35      35            35         35  
     IN        4     4      4       4             4          4  
...          ...   ...    ...     ...           ...        ...  
2021 CH      421   421    421     421           421        421  
     CU      277   277    277     277           277        277  
     FC      256   256    256     256           256        256  
     FF     1316  1316   1316    1316          1316       1316  
     SL      551   551    551     551           551        551  

[85 rows x 14 columns]

In [4]:
df_2019 = df[df['Year'] == 2019]

In [5]:
df_2019

,Unnamed: 0,Pitch,MPH,EV,Pitcher,Batter,Dist,Spin Rate,LA,Zone,Date,Count,Inning,Pitch Result,PA Result,Year
4034,4034,FF,92.7,76.7,"Scherzer, Max","Haseley, Adam",261.0,2299.0,39.0,6.0,2019-09-24,0-0,Top 6,hit_into_play,Adam Haseley flies out to center fielder Victo...,2019
4035,4035,CH,84.4,51.5,"Scherzer, Max","Kingery, Scott",3.0,1494.0,-24.0,9.0,2019-09-24,1-2,Top 6,hit_into_play,Scott Kingery singles on a soft ground ball to...,2019
4036,4036,FF,97.4,77.0,"Scherzer, Max","Kingery, Scott",203.0,2465.0,27.0,8.0,2019-09-24,1-2,Top 6,foul,Scott Kingery singles on a soft ground ball to...,2019
4037,4037,SL,84.8,NaN,"Scherzer, Max","Kingery, Scott",NaN,2295.0,NaN,14.0,2019-09-24,0-2,Top 6,ball,Scott Kingery singles on a soft ground ball to...,2019
4038,4038,CU,79.1,69.2,"Scherzer, Max","Kingery, Scott",163.0,2818.0,63.0,4.0,2019-09-24,0-1,Top 6,foul,Scott Kingery singles on a soft ground ball to...,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6799,6799,FF,95.5,NaN,"Scherzer, Max","Alonso, Pete",NaN,2632.0,NaN,12.0,2019-03-28,0-1,Top 1,ball,Pete Alonso strikes out on a foul tip.,2019
6800,6800,SL,85.6,NaN,"Scherzer, Max","Alonso, Pete",NaN,2370.0,NaN,6.0,2019-03-28,0-0,Top 1,swinging_strike,Pete Alonso strikes out on a foul tip.,2019
6801,6801,FF,96.3,NaN,"Scherzer, Max","Nimmo, Brandon",NaN,2738.0,NaN,5.0,2019-03-28,0-2,Top 1,swinging_strike,Brandon Nimmo strikes out swinging.,2019
6802,6802,FF,94.2,NaN,"Scherzer, Max","Nimmo, Brandon",NaN,2545.0,NaN,5.0,2019-03-28,0-1,Top 1,swinging_strike,Brandon Nimmo strikes out swinging.,2019


In [6]:
df_2019.to_csv('sch_2019.csv')